In [1]:
from astroNN.datasets import galaxy10sdss
from astroNN.datasets.galaxy10 import galaxy10cls_lookup
import pandas as pd
import seaborn as sns
import numpy as np
from keras import utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
import torchvision.transforms as transform




In [223]:
images, labels=galaxy10sdss.load_data()

images=images.astype(np.float32)
Gal_train, Gal_test, lab_train, lab_test=train_test_split(images, labels, test_size=0.2)
Gal_train=Gal_train/255.0
Gal_test=Gal_test/255.0

C:\Users\Alexb\.astroNN\datasets\Galaxy10.h5 was found!


In [224]:
Gal_train.shape, Gal_test.shape, lab_train.shape, lab_test.shape

((17428, 69, 69, 3), (4357, 69, 69, 3), (17428,), (4357,))

In [225]:

features=[x for x in galaxy10sdss.Galaxy10Class.values()]
    
for x in features:
    print(x)

Disk, Face-on, No Spiral
Smooth, Completely round
Smooth, in-between round
Smooth, Cigar shaped
Disk, Edge-on, Rounded Bulge
Disk, Edge-on, Boxy Bulge
Disk, Edge-on, No Bulge
Disk, Face-on, Tight Spiral
Disk, Face-on, Medium Spiral
Disk, Face-on, Loose Spiral


In [235]:
Ntransform=transform.Compose([transform.ToTensor()])

In [236]:
class Gal10Dataset(Dataset):
    def __init__(self, gal_data, labels, transform=None):
        self.gal_data=gal_data
        self.labels=labels
        self.transform=transform
    def __len__(self):
        return len(self.gal_data)
    def __getitem__(self,idx):
        sample=self.gal_data[idx]
        label=self.labels[idx]
        if self.transform:
            sample=self.transform(sample)
        return sample, label

In [238]:
train_set=Gal10Dataset(Gal_train, lab_train, transform=Ntransform)
test_set=Gal10Dataset(Gal_test,lab_test, transform=Ntransform)

imgs=torch.stack([img for img, _ in train_set], dim=0)
mean=imgs.view(1, -1).mean(dim=1)
std=imgs.view(1, -1).std(dim=1)
print(mean, std)

resize_norm=transform.Compose([transform.ToTensor(), transform.Resize((32, 32)), transform.Normalize(mean=mean, std=std)])

train_set=Gal10Dataset(Gal_train, lab_train, transform=resize_norm)
test_set=Gal10Dataset(Gal_test, lab_test, transform=resize_norm)

train_loader=DataLoader(train_set, batch_size=64, shuffle=True)
test_loader=DataLoader(test_set, batch_size=64, shuffle=True)

tensor([0.0909]) tensor([0.1266])


In [242]:
len(train_loader), len(test_loader)

(273, 69)

In [239]:
print(train_loader)
print(len(train_loader.dataset))
print(len(train_loader))
print(train_loader.batch_size)
for x, y in train_loader:
    print(x.shape, x.dtype, y.shape)
    break

17428
273
64
torch.Size([64, 3, 32, 32]) torch.float32 torch.Size([64])


In [240]:
for images, labels in train_loader:
    print(images.shape, images.dtype)
    print(labels.shape, labels.dtype)
    break

torch.Size([64, 3, 32, 32]) torch.float32
torch.Size([64]) torch.uint8


In [ ]:

fig, axs=plt.subplots(10, 2, figsize=(16,16))
for i in range(10):
    axs[i, 0].imshow(train_loader.dataset[i][0][0], cmap='gray')
    axs[i, 1].imshow(Gal_train[i], cmap='gray')

In [208]:
#The convolutional NN
import torch.nn as nn
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_class=num_classes
        self.feature=nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
        self.classifier=nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=16*5*5, out_features=120),
            nn.Tanh(),
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=self.num_class)
        )
        
    
    def forward(self, x):
        return self.classifier(self.feature(x))

In [209]:
device=(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

cuda


In [253]:
from torchinfo import summary
model=LeNet5(10)
summary(model=model, input_size=(1, 3, 32, 32), col_width=20,
        col_names=['input_size', 'output_size', 'num_params', 'trainable'], row_settings=['var_names'], verbose=0)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
LeNet5 (LeNet5)                          [1, 3, 32, 32]       [1, 10]              --                   True
├─Sequential (feature)                   [1, 3, 32, 32]       [1, 16, 5, 5]        --                   True
│    └─Conv2d (0)                        [1, 3, 32, 32]       [1, 6, 28, 28]       456                  True
│    └─Tanh (1)                          [1, 6, 28, 28]       [1, 6, 28, 28]       --                   --
│    └─AvgPool2d (2)                     [1, 6, 28, 28]       [1, 6, 14, 14]       --                   --
│    └─Conv2d (3)                        [1, 6, 14, 14]       [1, 16, 10, 10]      2,416                True
│    └─Tanh (4)                          [1, 16, 10, 10]      [1, 16, 10, 10]      --                   --
│    └─AvgPool2d (5)                     [1, 16, 10, 10]      [1, 16, 5, 5]        --                   --
├─Sequential (classifi

In [189]:
num_features=len(features)
learning_rate=0.001
epochs=10

model=LeNet5(num_classes=num_features).to(device=device)
loss_fn=nn.CrossEntropyLoss()

optim=torch.optim.Adam(model.parameters(),lr=learning_rate)

total_steps=len(train_loader)